# Import necessary Modules

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import scipy
from scipy import stats
from scipy.stats import norm
import datetime as dt
from sklearn import linear_model
import operator
import sklearn_pandas
from sklearn import preprocessing
from sklearn.preprocessing import Imputer
from sklearn_pandas import CategoricalImputer
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Dense
from sklearn.ensemble import RandomForestClassifier as RFC
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE 
from keras.models import load_model
import pickle
from scipy.stats import chi2_contingency

c:\users\msros\anaconda22\envs\py36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


# Load the Data

In [2]:
liver_df = pd.read_csv('liver_data_inf560.csv')
print (liver_df.shape)


(84603, 144)


# Clean the Data

In [3]:
#remove rows where age<18
liver_df = liver_df.loc[liver_df['AGE'] >=18]

#remove rows where ptime<gtime
liver_df = liver_df.loc[liver_df['GTIME'] < liver_df['PTIME']]


In [4]:
post_transplant_features=['ACUTE_REJ_EPI','GRF_STAT','GTIME','PTIME','PSTATUS','PX_STAT']
liver_df.drop(post_transplant_features, axis=1, inplace=True)


In [5]:
X_cf = liver_df.select_dtypes(include=['object'])
print (X_cf.shape)
X_ncf = liver_df.select_dtypes(exclude=['object'])
print (X_ncf.shape)


(39162, 68)
(39162, 70)


# Handle Null Values for Categorical Features

In [6]:
imp_cat = CategoricalImputer()
X_cf = pd.DataFrame(imp_cat.fit_transform(np.array(X_cf)),columns = X_cf.columns)

# Matching score for categorical features

In [7]:
X_cf_gstatus0 = liver_df.loc[liver_df['GSTATUS'] == 0]
X_cf_gstatus1 = liver_df.loc[liver_df['GSTATUS'] == 1]

#Recipient features
tab = pd.crosstab(liver_df['ABO'], liver_df['GSTATUS'])
a,b,c,d = chi2_contingency(tab.values)
print(a,b)

tab = pd.crosstab(liver_df['GENDER'], liver_df['GSTATUS'])
a,b,c,d = chi2_contingency(tab.values)
print(a,b)

tab = pd.crosstab(liver_df['HBV_CORE'], liver_df['GSTATUS'])
a,b,c,d = chi2_contingency(tab.values)
print(a,b)

tab = pd.crosstab(liver_df['HBV_SUR_ANTIGEN'], liver_df['GSTATUS'])
a,b,c,d = chi2_contingency(tab.values)
print(a,b)


#Donor features
tab = pd.crosstab(liver_df['ABO_DON'], liver_df['GSTATUS'])
a,b,c,d = chi2_contingency(tab.values)
print(a,b)

tab = pd.crosstab(liver_df['GENDER_DON'], liver_df['GSTATUS'])
a,b,c,d = chi2_contingency(tab.values)
print(a,b)

tab = pd.crosstab(liver_df['HBV_CORE_DON'], liver_df['GSTATUS'])
a,b,c,d = chi2_contingency(tab.values)
print(a,b)

tab = pd.crosstab(liver_df['HBV_SUR_ANTIGEN_DON'], liver_df['GSTATUS'])
a,b,c,d = chi2_contingency(tab.values)
print(a,b)


weight_cat_norm = [1,0.0183,0.3499,0.6673]
recipient_class_gstatus0 = np.array(X_cf_gstatus0[['ABO','GENDER','HBV_CORE','HBV_SUR_ANTIGEN']])
recipient_class_gstatus1 = np.array(X_cf_gstatus1[['ABO','GENDER','HBV_CORE','HBV_SUR_ANTIGEN']])

donor_class_gstatus0 = np.array(X_cf_gstatus0[['ABO_DON','GENDER_DON','HBV_CORE_DON','HBV_SUR_ANTIGEN_DON']])
donor_class_gstatus1 = np.array(X_cf_gstatus1[['ABO_DON','GENDER_DON','HBV_CORE_DON','HBV_SUR_ANTIGEN_DON']])

cat_weights_gstatus0 = []
cat_weights_gstatus1 = []
for i in range(len(recipient_class_gstatus0)):
    weights = 0
    for j in range(len(recipient_class_gstatus0[i])):
        if recipient_class_gstatus0[i][j] == donor_class_gstatus0[i][j]:
            weights += weight_cat_norm[j]
    cat_weights_gstatus0.append(weights)



for i in range(len(recipient_class_gstatus1)):
    weights = 0
    for j in range(len(recipient_class_gstatus1[i])):
        if recipient_class_gstatus1[i][j] == donor_class_gstatus1[i][j]:
            weights += weight_cat_norm[j]
    cat_weights_gstatus1.append(weights)



10.409352104886752 0.1665380663371406
0.06282071537955335 0.8020919814141305
0.44958680287655994 0.929818855434263
4.519254637301104 0.21057923438654064
6.57304278863462 0.4746439448790196
0.06943263592455594 0.7921646591920652
5.493224660679154 0.2403256795930887
6.814478096652416 0.23480747664855348


# One Hot Encoding for Categorical Features

In [8]:
X_cfn=pd.get_dummies(X_cf)
print(X_cfn.shape)
X_cfn.head()

(39162, 208)


,GENDER_F,GENDER_M,ABO_A,ABO_A1,ABO_A1B,ABO_A2,ABO_A2B,ABO_AB,ABO_B,ABO_O,...,HISTORY_MI_DON_N,HISTORY_MI_DON_U,HISTORY_MI_DON_Y,CORONARY_ANGIO_DON_N,CORONARY_ANGIO_DON_Y,LIST_MELD_No,TX_MELD_No,LT_ONE_WEEK_DON_N,DATA_WAITLIST_Y,DATA_TRANSPLANT_Y
0,0,1,1,0,0,0,0,0,0,0,...,1,0,0,1,0,1,1,1,1,1
1,0,1,0,0,0,0,0,0,0,1,...,1,0,0,1,0,1,1,1,1,1
2,1,0,0,0,0,0,0,0,1,0,...,1,0,0,1,0,1,1,1,1,1
3,0,1,0,0,0,0,0,0,1,0,...,1,0,0,1,0,1,1,1,1,1
4,0,1,0,0,0,0,0,0,0,1,...,1,0,0,1,0,1,1,1,1,1


# Handle Null Values for Non - Categorical Features

In [9]:
#Non categorical features
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(X_ncf)
X_ncf = pd.DataFrame(imp.transform(X_ncf),columns = X_ncf.columns)


In [27]:
list1=[23,1,6,8,0]

arr1=np.array(list1)
print (arr1)
print (list(arr1.argsort())[::-1][:5])

[23  1  6  8  0]
[0, 3, 2, 1, 4]


# Preprocess the Data

In [10]:
min_max_scaler = preprocessing.MinMaxScaler()
header=X_ncf.columns
X_train_minmax = min_max_scaler.fit_transform(X_ncf)
X_ncf=pd.DataFrame(X_train_minmax,columns=header)

In [11]:
#merge categorical and non-categorical features
result = pd.merge(X_ncf, X_cfn,left_index=True,right_index=True)
print(result.shape)


(39162, 278)


# Matching for continuous features

In [12]:
#Recepients
liver_df_gstatus0 = result.loc[result['GSTATUS'] == 0]
liver_df_gstatus1 = result.loc[result['GSTATUS'] == 1]

liver_df1 = liver_df_gstatus0.dropna(subset=['BMI_CALC'])
liver_df2 = liver_df_gstatus1.dropna(subset=['BMI_CALC'])
print(scipy.stats.ttest_ind(liver_df1['BMI_CALC'],liver_df2['BMI_CALC']))

liver_df1 = liver_df_gstatus0.dropna(subset=['CREAT_TX'])
liver_df2 = liver_df_gstatus1.dropna(subset=['CREAT_TX'])
print(scipy.stats.ttest_ind(liver_df1['CREAT_TX'],liver_df2['CREAT_TX']))

liver_df1 = liver_df_gstatus0.dropna(subset=['TBILI_TX'])
liver_df2 = liver_df_gstatus1.dropna(subset=['TBILI_TX'])
print(scipy.stats.ttest_ind(liver_df1['TBILI_TX'],liver_df2['TBILI_TX']))

liver_df1 = liver_df_gstatus0.dropna(subset=['HGT_CM_CALC'])
liver_df2 = liver_df_gstatus1.dropna(subset=['HGT_CM_CALC'])
print(scipy.stats.ttest_ind(liver_df1['HGT_CM_CALC'],liver_df2['HGT_CM_CALC']))

liver_df1 = liver_df_gstatus0.dropna(subset=['WGT_KG_CALC'])
liver_df2 = liver_df_gstatus1.dropna(subset=['WGT_KG_CALC'])
print(scipy.stats.ttest_ind(liver_df1['WGT_KG_CALC'],liver_df2['WGT_KG_CALC']))

weight_cat = [16.9823,0.1322,5.9427,11.3336]
weight_cat_norm = [1,0.0183,0.3499,0.6673]
recipient_weights_cont = [0.8926*10,1*10,0.3142*10,0.7883,0.3940*10]



Ttest_indResult(statistic=5.4620759287569625, pvalue=4.7345693336526015e-08)
Ttest_indResult(statistic=-6.118624127626865, pvalue=9.527954947230177e-10)
Ttest_indResult(statistic=-1.922643877434707, pvalue=0.05453203429231182)
Ttest_indResult(statistic=-4.823512603757652, pvalue=1.4158132010364828e-06)
Ttest_indResult(statistic=2.410839649330238, pvalue=0.015920419504155896)


In [13]:
#Donors


liver_df1 = liver_df_gstatus0.dropna(subset=['BMI_DON_CALC'])
liver_df2 = liver_df_gstatus1.dropna(subset=['BMI_DON_CALC'])
print(scipy.stats.ttest_ind(liver_df1['BMI_DON_CALC'],liver_df2['BMI_DON_CALC']))

liver_df1 = liver_df_gstatus0.dropna(subset=['CREAT_DON'])
liver_df2 = liver_df_gstatus1.dropna(subset=['CREAT_DON'])
print(scipy.stats.ttest_ind(liver_df1['CREAT_DON'],liver_df2['CREAT_DON']))

liver_df1 = liver_df_gstatus0.dropna(subset=['TBILI_DON'])
liver_df2 = liver_df_gstatus1.dropna(subset=['TBILI_DON'])
print(scipy.stats.ttest_ind(liver_df1['TBILI_DON'],liver_df2['TBILI_DON']))

liver_df1 = liver_df_gstatus0.dropna(subset=['HGT_CM_DON_CALC'])
liver_df2 = liver_df_gstatus1.dropna(subset=['HGT_CM_DON_CALC'])
print(scipy.stats.ttest_ind(liver_df1['HGT_CM_DON_CALC'],liver_df2['HGT_CM_DON_CALC']))

liver_df1 = liver_df_gstatus0.dropna(subset=['WGT_KG_DON_CALC'])
liver_df2 = liver_df_gstatus1.dropna(subset=['WGT_KG_DON_CALC'])
print(scipy.stats.ttest_ind(liver_df1['WGT_KG_DON_CALC'],liver_df2['WGT_KG_DON_CALC']))


donor_weights_cont = [0.4466*10,0.1846*10,0.2730*10,0.8203*10,0.7958*10]


Ttest_indResult(statistic=2.7330502594820385, pvalue=0.006277891372224274)
Ttest_indResult(statistic=1.1295205067656648, pvalue=0.2586852425321753)
Ttest_indResult(statistic=-1.670970443453076, pvalue=0.09473550667059624)
Ttest_indResult(statistic=5.019333630605845, pvalue=5.20779640848788e-07)
Ttest_indResult(statistic=4.869530386776263, pvalue=1.1229933706501736e-06)


# Computing score for continuous features

In [14]:
recepient_cont_gstatus0 = np.array(liver_df_gstatus0[['BMI_CALC','CREAT_TX','TBILI_TX','HGT_CM_CALC','WGT_KG_CALC']])
recepient_cont_gstatus1 = np.array(liver_df_gstatus1[['BMI_CALC','CREAT_TX','TBILI_TX','HGT_CM_CALC','WGT_KG_CALC']])

donor_cont_gstatus0 = np.array(liver_df_gstatus0[['BMI_DON_CALC','CREAT_DON','TBILI_DON','HGT_CM_DON_CALC','WGT_KG_DON_CALC']])
donor_cont_gstatus1 = np.array(liver_df_gstatus1[['BMI_DON_CALC','CREAT_DON','TBILI_DON','HGT_CM_DON_CALC','WGT_KG_DON_CALC']])

dist_gstatus_0 = np.sum(((recipient_weights_cont*recepient_cont_gstatus0)-(donor_weights_cont*donor_cont_gstatus0))**2,axis=-1, keepdims=True)**0.5
dist_gstatus_1 = np.sum(((recipient_weights_cont*recepient_cont_gstatus1)-(donor_weights_cont*donor_cont_gstatus1))**2,axis=-1, keepdims=True)**0.5

print(np.shape(dist_gstatus_0))



(27505, 1)


# Computing score for categorical features

In [15]:
#Categorical score
cat_weights_gstatus_1 = np.array(cat_weights_gstatus1)
cat_weights_gstatus_0 = np.array(cat_weights_gstatus0)


# Final Matching score

In [16]:
final_gstatus_0 = []
final_gstatus_1 = []

for i in range(len(dist_gstatus_0)):
        final_gstatus_0.append(dist_gstatus_0[i]+cat_weights_gstatus_0[i])
        
for i in range(len(dist_gstatus_1)):
        final_gstatus_1.append(dist_gstatus_1[i]+cat_weights_gstatus_1[i])
        


# Computing the difference between the mean of two distributions (gstatus =0 and1)

In [17]:
print(scipy.stats.ttest_ind(np.array(final_gstatus_0),np.array(final_gstatus_1)))

Ttest_indResult(statistic=array([4.61071389]), pvalue=array([4.02554922e-06]))


# X and Y to fit the model

In [18]:
less_important_features=['ABO_DON_A2B','ABO_DON_AB','ANTIHYPE_DON_Y','CARDARREST_NEURO_N','CARDARREST_NEURO_Y','CDC_RISK_HIV_DON_U','CLIN_INFECT_DON_U','DATA_TRANSPLANT_Y','DATA_WAITLIST_Y','DIABETES_DON_U','DIAL_TX_N','DIAL_TX_Y','EXTRACRANIAL_CANCER_DON_Y','HBV_CORE_DON.1_U','HBV_CORE_DON_I','HBV_CORE_ND','HBV_CORE_U','HBV_SUR_ANTIGEN_DON_C','HBV_SUR_ANTIGEN_DON_I','HCV_SEROSTATUS_ND','HEP_C_ANTI_DON_C','HEP_C_ANTI_DON_I','HISTORY_MI_DON_N','HIST_CANCER_DON_U','HIST_OTH_DRUG_DON_U','HIV_SEROSTATUS_ND','HIV_SEROSTATUS_P','INSULIN_DON_Y','LIFE_SUP_TRR_N','LIFE_SUP_TRR_Y','LIST_MELD_No','LT_ONE_WEEK_DON_N','MALIG_TRR_Y','PT_T3_DON_N','PT_T3_DON_Y','SKIN_CANCER_DON_N','TXLIV_S','TXLIV_W','TX_MELD_No','VDRL_DON_I']
cols = [col for col in result.columns if col not in less_important_features]
X= result[cols]
X = X[[column for column in X.columns if column != 'GSTATUS']]

Y= result[['GSTATUS']]
#print (list(X.columns))
headers= ['NUM_PREV_TX', 'DIAB', 'REM_CD', 'DAYSWAIT_CHRON', 'END_STAT', 'END_BMI_CALC', 'FINAL_ALBUMIN', 'FINAL_ASCITES', 'FINAL_BILIRUBIN', 'FINAL_ENCEPH', 'FINAL_INR', 'FINAL_MELD_PELD_LAB_SCORE', 'FINAL_SERUM_CREAT', 'FINAL_SERUM_SODIUM', 'TX_PROCEDUR_TY', 'FUNC_STAT_TRR', 'MED_COND_TRR', 'PRI_PAYMENT_TRR', 'ON_VENT_TRR', 'ARTIFICIAL_LI_TRR', 'OTH_LIFE_SUP_TRR', 'DA1', 'DA2', 'DB1', 'DB2', 'DDR1', 'DDR2', 'AGE_DON', 'COD_CAD_DON', 'DEATH_CIRCUM_DON', 'DEATH_MECH_DON', 'BLOOD_INF_DON', 'BUN_DON', 'CREAT_DON', 'PULM_INF_DON', 'SGOT_DON', 'SGPT_DON', 'TBILI_DON', 'URINE_INF_DON', 'CANCER_SITE_DON', 'HIST_DIABETES_DON', 'HGT_CM_DON_CALC', 'WGT_KG_DON_CALC', 'BMI_DON_CALC', 'ECD_DONOR', 'CREAT_TX', 'TBILI_TX', 'INR_TX', 'ALBUMIN_TX', 'ENCEPH_TX', 'ASCITES_TX', 'MELD_PELD_LAB_SCORE', 'EXC_EVER', 'LITYP', 'LOS', 'AGE', 'DIAG', 'ABO_MAT', 'COLD_ISCH', 'SHARE_TY', 'HGT_CM_CALC', 'WGT_KG_CALC', 'BMI_CALC', 'DONOR_ID', 'TRANSFUS_TERM_DON', 'PH_DON', 'HEMATOCRIT_DON', 'TX_YEAR', 'LISTYR', 'GENDER_F', 'GENDER_M', 'ABO_A', 'ABO_A1', 'ABO_A1B', 'ABO_A2', 'ABO_A2B', 'ABO_AB', 'ABO_B', 'ABO_O', 'EXC_HCC_HBL', 'EXC_HCC_HCC', 'EXC_HCC_non-HCC', 'EXC_CASE_No', 'EXC_CASE_Yes', 'FINAL_DIALYSIS_PRIOR_WEEK_A', 'FINAL_DIALYSIS_PRIOR_WEEK_N', 'FINAL_DIALYSIS_PRIOR_WEEK_Y', 'FINAL_MELD_OR_PELD_MELD', 'FINAL_MELD_OR_PELD_PELD', 'MALIG_TRR_N', 'MALIG_TRR_U', 'PORTAL_VEIN_TRR_N', 'PORTAL_VEIN_TRR_U', 'PORTAL_VEIN_TRR_Y', 'PREV_AB_SURG_TRR_N', 'PREV_AB_SURG_TRR_U', 'PREV_AB_SURG_TRR_Y', 'TIPSS_TRR_N', 'TIPSS_TRR_U', 'TIPSS_TRR_Y', 'HBV_CORE_N', 'HBV_CORE_P', 'HBV_SUR_ANTIGEN_N', 'HBV_SUR_ANTIGEN_ND', 'HBV_SUR_ANTIGEN_P', 'HBV_SUR_ANTIGEN_U', 'HCV_SEROSTATUS_N', 'HCV_SEROSTATUS_P', 'HCV_SEROSTATUS_U', 'EBV_SEROSTATUS_N', 'EBV_SEROSTATUS_ND', 'EBV_SEROSTATUS_P', 'EBV_SEROSTATUS_U', 'HIV_SEROSTATUS_N', 'HIV_SEROSTATUS_U', 'CMV_STATUS_N', 'CMV_STATUS_ND', 'CMV_STATUS_P', 'CMV_STATUS_U', 'CMV_IGG_N', 'CMV_IGG_ND', 'CMV_IGG_P', 'CMV_IGG_U', 'CMV_IGM_N', 'CMV_IGM_ND', 'CMV_IGM_P', 'CMV_IGM_U', 'PREV_TX_N', 'PREV_TX_Y', 'DDAVP_DON_N', 'DDAVP_DON_U', 'DDAVP_DON_Y', 'CMV_DON_I', 'CMV_DON_N', 'CMV_DON_ND', 'CMV_DON_P', 'CMV_DON_U', 'HEP_C_ANTI_DON_N', 'HEP_C_ANTI_DON_ND', 'HEP_C_ANTI_DON_P', 'HEP_C_ANTI_DON_U', 'HBV_CORE_DON_N', 'HBV_CORE_DON_ND', 'HBV_CORE_DON_P', 'HBV_CORE_DON_U', 'HBV_SUR_ANTIGEN_DON_N', 'HBV_SUR_ANTIGEN_DON_ND', 'HBV_SUR_ANTIGEN_DON_P', 'HBV_SUR_ANTIGEN_DON_U', 'DON_TY_C', 'DON_TY_L', 'GENDER_DON_F', 'GENDER_DON_M', 'NON_HRT_DON_N', 'NON_HRT_DON_Y', 'ANTIHYPE_DON_N', 'ANTIHYPE_DON_U', 'PT_DIURETICS_DON_N', 'PT_DIURETICS_DON_U', 'PT_DIURETICS_DON_Y', 'PT_STEROIDS_DON_N', 'PT_STEROIDS_DON_U', 'PT_STEROIDS_DON_Y', 'PT_T3_DON_U', 'PT_T4_DON_N', 'PT_T4_DON_U', 'PT_T4_DON_Y', 'VASODIL_DON_N', 'VASODIL_DON_U', 'VASODIL_DON_Y', 'VDRL_DON_N', 'VDRL_DON_ND', 'VDRL_DON_P', 'VDRL_DON_U', 'CLIN_INFECT_DON_N', 'CLIN_INFECT_DON_Y', 'EXTRACRANIAL_CANCER_DON_N', 'EXTRACRANIAL_CANCER_DON_U', 'HIST_CIG_DON_N', 'HIST_CIG_DON_U', 'HIST_CIG_DON_Y', 'HIST_COCAINE_DON_N', 'HIST_COCAINE_DON_U', 'HIST_COCAINE_DON_Y', 'DIABETES_DON_N', 'DIABETES_DON_Y', 'HIST_HYPERTENS_DON_N', 'HIST_HYPERTENS_DON_U', 'HIST_HYPERTENS_DON_Y', 'HIST_OTH_DRUG_DON_N', 'HIST_OTH_DRUG_DON_Y', 'ABO_DON_A', 'ABO_DON_A1', 'ABO_DON_A1B', 'ABO_DON_A2', 'ABO_DON_B', 'ABO_DON_O', 'INTRACRANIAL_CANCER_DON_N', 'INTRACRANIAL_CANCER_DON_U', 'INTRACRANIAL_CANCER_DON_Y', 'SKIN_CANCER_DON_U', 'SKIN_CANCER_DON_Y', 'HIST_CANCER_DON_N', 'HIST_CANCER_DON_Y', 'PT_OTH_DON_N', 'PT_OTH_DON_Y', 'HEPARIN_DON_N', 'HEPARIN_DON_U', 'HEPARIN_DON_Y', 'HBV_CORE_DON.1_N', 'HBV_CORE_DON.1_Y', 'INSULIN_DON_N', 'INSULIN_DON_U', 'AGE_GROUP_A', 'AGE_GROUP_P', 'MALIG_N', 'MALIG_U', 'MALIG_Y', 'RECOV_OUT_US_N', 'RECOV_OUT_US_Y', 'TATTOOS_N', 'TATTOOS_U', 'TATTOOS_Y', 'LI_BIOPSY_N', 'LI_BIOPSY_Y', 'PROTEIN_URINE_N', 'PROTEIN_URINE_U', 'PROTEIN_URINE_Y', 'INOTROP_SUPPORT_DON_N', 'INOTROP_SUPPORT_DON_U', 'INOTROP_SUPPORT_DON_Y', 'CDC_RISK_HIV_DON_N', 'CDC_RISK_HIV_DON_Y', 'HISTORY_MI_DON_U', 'HISTORY_MI_DON_Y', 'CORONARY_ANGIO_DON_N', 'CORONARY_ANGIO_DON_Y']

x_train=X
y_train=Y

print(x_train.shape)
print(y_train.shape)
display(x_train)
display(y_train)

(39162, 237)
(39162, 1)


,NUM_PREV_TX,DIAB,REM_CD,DAYSWAIT_CHRON,END_STAT,END_BMI_CALC,FINAL_ALBUMIN,FINAL_ASCITES,FINAL_BILIRUBIN,FINAL_ENCEPH,...,PROTEIN_URINE_Y,INOTROP_SUPPORT_DON_N,INOTROP_SUPPORT_DON_U,INOTROP_SUPPORT_DON_Y,CDC_RISK_HIV_DON_N,CDC_RISK_HIV_DON_Y,HISTORY_MI_DON_U,HISTORY_MI_DON_Y,CORONARY_ANGIO_DON_N,CORONARY_ANGIO_DON_Y
0,0.000000,0.002006,1.000000,0.011615,0.217391,0.280124,0.287234,0.666667,0.041237,0.000000,...,0,1,0,0,1,0,0,0,1,0
1,0.000000,0.000000,0.666667,0.002050,0.216380,0.415921,0.180851,0.333333,0.039175,0.000000,...,1,0,0,1,1,0,0,0,1,0
2,0.000000,0.000000,0.055556,0.128445,0.215369,0.244989,0.372340,0.333333,0.170103,0.333333,...,0,1,0,0,1,0,0,0,1,0
3,0.000000,0.000000,0.055556,0.002733,0.225480,0.332678,0.191489,0.666667,0.041237,0.333333,...,1,1,0,0,1,0,0,0,1,0
4,0.000000,0.000000,0.055556,0.008199,0.219414,0.095644,0.223404,0.333333,0.054639,0.666667,...,0,1,0,0,1,0,0,0,1,0
5,0.000000,0.000000,0.055556,0.001366,0.002022,0.153418,0.393617,0.666667,0.005155,0.000000,...,1,0,0,1,1,0,0,0,0,1
6,0.000000,0.000000,0.055556,0.002505,0.230536,0.208254,0.351064,0.333333,0.011340,0.000000,...,0,1,0,0,1,0,0,0,1,0
7,0.000000,0.002006,0.055556,0.003872,0.227503,0.322488,0.255319,0.333333,0.050515,0.333333,...,0,1,0,0,1,0,0,0,1,0
8,0.111111,0.000000,0.055556,0.003872,0.216380,0.395840,0.223404,0.666667,0.094021,0.333333,...,0,1,0,0,1,0,0,0,1,0
9,0.000000,0.002006,0.055556,0.046686,0.222447,0.200581,0.202128,0.333333,0.030928,0.000000,...,0,1,0,0,1,0,0,0,1,0


,GSTATUS
0,1.0
1,1.0
2,1.0
3,0.0
4,1.0
5,1.0
6,0.0
7,0.0
8,0.0
9,0.0


# Split the Data into Training and Test Data

x_train, x_test,y_train,y_test = train_test_split(X,Y, test_size = 0.30, random_state=4)


In [19]:
xtrain_columns =['NUM_PREV_TX', 'DIAB', 'REM_CD', 'DAYSWAIT_CHRON', 'END_STAT', 'END_BMI_CALC', 'FINAL_ALBUMIN', 'FINAL_ASCITES', 'FINAL_BILIRUBIN', 'FINAL_ENCEPH', 'FINAL_INR', 'FINAL_MELD_PELD_LAB_SCORE', 'FINAL_SERUM_CREAT', 'FINAL_SERUM_SODIUM', 'TX_PROCEDUR_TY', 'FUNC_STAT_TRR', 'MED_COND_TRR', 'PRI_PAYMENT_TRR', 'ON_VENT_TRR', 'ARTIFICIAL_LI_TRR', 'OTH_LIFE_SUP_TRR', 'DA1', 'DA2', 'DB1', 'DB2', 'DDR1', 'DDR2', 'AGE_DON', 'COD_CAD_DON', 'DEATH_CIRCUM_DON', 'DEATH_MECH_DON', 'BLOOD_INF_DON', 'BUN_DON', 'CREAT_DON', 'PULM_INF_DON', 'SGOT_DON', 'SGPT_DON', 'TBILI_DON', 'URINE_INF_DON', 'CANCER_SITE_DON', 'HIST_DIABETES_DON', 'HGT_CM_DON_CALC', 'WGT_KG_DON_CALC', 'BMI_DON_CALC', 'ECD_DONOR', 'CREAT_TX', 'TBILI_TX', 'INR_TX', 'ALBUMIN_TX', 'ENCEPH_TX', 'ASCITES_TX', 'MELD_PELD_LAB_SCORE', 'EXC_EVER', 'LITYP', 'LOS', 'AGE', 'DIAG', 'ABO_MAT', 'COLD_ISCH', 'SHARE_TY', 'HGT_CM_CALC', 'WGT_KG_CALC', 'BMI_CALC', 'DONOR_ID', 'TRANSFUS_TERM_DON', 'PH_DON', 'HEMATOCRIT_DON', 'TX_YEAR', 'LISTYR', 'GENDER_F', 'GENDER_M', 'ABO_A', 'ABO_A1', 'ABO_A1B', 'ABO_A2', 'ABO_A2B', 'ABO_AB', 'ABO_B', 'ABO_O', 'EXC_HCC_HBL', 'EXC_HCC_HCC', 'EXC_HCC_non-HCC', 'EXC_CASE_No', 'EXC_CASE_Yes', 'FINAL_DIALYSIS_PRIOR_WEEK_A', 'FINAL_DIALYSIS_PRIOR_WEEK_N', 'FINAL_DIALYSIS_PRIOR_WEEK_Y', 'FINAL_MELD_OR_PELD_MELD', 'FINAL_MELD_OR_PELD_PELD', 'MALIG_TRR_N', 'MALIG_TRR_U', 'PORTAL_VEIN_TRR_N', 'PORTAL_VEIN_TRR_U', 'PORTAL_VEIN_TRR_Y', 'PREV_AB_SURG_TRR_N', 'PREV_AB_SURG_TRR_U', 'PREV_AB_SURG_TRR_Y', 'TIPSS_TRR_N', 'TIPSS_TRR_U', 'TIPSS_TRR_Y', 'HBV_CORE_N', 'HBV_CORE_P', 'HBV_SUR_ANTIGEN_N', 'HBV_SUR_ANTIGEN_ND', 'HBV_SUR_ANTIGEN_P', 'HBV_SUR_ANTIGEN_U', 'HCV_SEROSTATUS_N', 'HCV_SEROSTATUS_P', 'HCV_SEROSTATUS_U', 'EBV_SEROSTATUS_N', 'EBV_SEROSTATUS_ND', 'EBV_SEROSTATUS_P', 'EBV_SEROSTATUS_U', 'HIV_SEROSTATUS_N', 'HIV_SEROSTATUS_U', 'CMV_STATUS_N', 'CMV_STATUS_ND', 'CMV_STATUS_P', 'CMV_STATUS_U', 'CMV_IGG_N', 'CMV_IGG_ND', 'CMV_IGG_P', 'CMV_IGG_U', 'CMV_IGM_N', 'CMV_IGM_ND', 'CMV_IGM_P', 'CMV_IGM_U', 'PREV_TX_N', 'PREV_TX_Y', 'DDAVP_DON_N', 'DDAVP_DON_U', 'DDAVP_DON_Y', 'CMV_DON_I', 'CMV_DON_N', 'CMV_DON_ND', 'CMV_DON_P', 'CMV_DON_U', 'HEP_C_ANTI_DON_N', 'HEP_C_ANTI_DON_ND', 'HEP_C_ANTI_DON_P', 'HEP_C_ANTI_DON_U', 'HBV_CORE_DON_N', 'HBV_CORE_DON_ND', 'HBV_CORE_DON_P', 'HBV_CORE_DON_U', 'HBV_SUR_ANTIGEN_DON_N', 'HBV_SUR_ANTIGEN_DON_ND', 'HBV_SUR_ANTIGEN_DON_P', 'HBV_SUR_ANTIGEN_DON_U', 'DON_TY_C', 'DON_TY_L', 'GENDER_DON_F', 'GENDER_DON_M', 'NON_HRT_DON_N', 'NON_HRT_DON_Y', 'ANTIHYPE_DON_N', 'ANTIHYPE_DON_U', 'PT_DIURETICS_DON_N', 'PT_DIURETICS_DON_U', 'PT_DIURETICS_DON_Y', 'PT_STEROIDS_DON_N', 'PT_STEROIDS_DON_U', 'PT_STEROIDS_DON_Y', 'PT_T3_DON_U', 'PT_T4_DON_N', 'PT_T4_DON_U', 'PT_T4_DON_Y', 'VASODIL_DON_N', 'VASODIL_DON_U', 'VASODIL_DON_Y', 'VDRL_DON_N', 'VDRL_DON_ND', 'VDRL_DON_P', 'VDRL_DON_U', 'CLIN_INFECT_DON_N', 'CLIN_INFECT_DON_Y', 'EXTRACRANIAL_CANCER_DON_N', 'EXTRACRANIAL_CANCER_DON_U', 'HIST_CIG_DON_N', 'HIST_CIG_DON_U', 'HIST_CIG_DON_Y', 'HIST_COCAINE_DON_N', 'HIST_COCAINE_DON_U', 'HIST_COCAINE_DON_Y', 'DIABETES_DON_N', 'DIABETES_DON_Y', 'HIST_HYPERTENS_DON_N', 'HIST_HYPERTENS_DON_U', 'HIST_HYPERTENS_DON_Y', 'HIST_OTH_DRUG_DON_N', 'HIST_OTH_DRUG_DON_Y', 'ABO_DON_A', 'ABO_DON_A1', 'ABO_DON_A1B', 'ABO_DON_A2', 'ABO_DON_B', 'ABO_DON_O', 'INTRACRANIAL_CANCER_DON_N', 'INTRACRANIAL_CANCER_DON_U', 'INTRACRANIAL_CANCER_DON_Y', 'SKIN_CANCER_DON_U', 'SKIN_CANCER_DON_Y', 'HIST_CANCER_DON_N', 'HIST_CANCER_DON_Y', 'PT_OTH_DON_N', 'PT_OTH_DON_Y', 'HEPARIN_DON_N', 'HEPARIN_DON_U', 'HEPARIN_DON_Y', 'HBV_CORE_DON.1_N', 'HBV_CORE_DON.1_Y', 'INSULIN_DON_N', 'INSULIN_DON_U', 'AGE_GROUP_A', 'AGE_GROUP_P', 'MALIG_N', 'MALIG_U', 'MALIG_Y', 'RECOV_OUT_US_N', 'RECOV_OUT_US_Y', 'TATTOOS_N', 'TATTOOS_U', 'TATTOOS_Y', 'LI_BIOPSY_N', 'LI_BIOPSY_Y', 'PROTEIN_URINE_N', 'PROTEIN_URINE_U', 'PROTEIN_URINE_Y', 'INOTROP_SUPPORT_DON_N', 'INOTROP_SUPPORT_DON_U', 'INOTROP_SUPPORT_DON_Y', 'CDC_RISK_HIV_DON_N', 'CDC_RISK_HIV_DON_Y', 'HISTORY_MI_DON_U', 'HISTORY_MI_DON_Y', 'CORONARY_ANGIO_DON_N', 'CORONARY_ANGIO_DON_Y']

#x_test= read from csv, convert to a dataframe, name it x_test
x_test= pd.read_csv('x_test.csv')

def add_missing_dummy_columns( x_test, xtrain_columns ):
    missing_cols = set(xtrain_columns) - set(x_test.columns)
    for c in missing_cols:
        x_test[c] = 0
        
def fix_columns(x_test, xtrain_columns ):  
    add_missing_dummy_columns( x_test, xtrain_columns )
    # make sure we have all the columns we need
    assert( set( xtrain_columns ) - set( x_test.columns ) == set())
    extra_cols = set( x_test.columns ) - set( xtrain_columns )
    x_test = x_test[ xtrain_columns ]
    return x_test

#Identify categorical and non-categorical
x_test_cf = x_test.select_dtypes(include=['object'])
x_test_ncf = x_test.select_dtypes(exclude=['object'])

#Handle NULL values in categorical features
imp_cat = CategoricalImputer()
x_test_cf = pd.DataFrame(imp_cat.fit_transform(np.array(x_test_cf)),columns = x_test_cf.columns)

#One-hot encoding for categorical features
x_test_cfn=pd.get_dummies(x_test_cf)

#Handle NULL values in non-categorical features
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(x_test_ncf)
x_test_ncf = pd.DataFrame(imp.transform(x_test_ncf),columns = x_test_ncf.columns)

#Scale test data
min_max_scaler = preprocessing.MinMaxScaler()
header=x_test_ncf.columns
x_test_minmax = min_max_scaler.fit_transform(x_test_ncf)
x_test_ncf=pd.DataFrame(x_test_minmax,columns=header)

#merge categorical and non-categorical features
result = pd.merge(x_test_ncf, x_test_cfn,left_index=True,right_index=True)
#fix columns
result=fix_columns(result,xtrain_columns)

#Eliminate less important features
less_important_features=['ABO_DON_A2B','ABO_DON_AB','ANTIHYPE_DON_Y','CARDARREST_NEURO_N','CARDARREST_NEURO_Y','CDC_RISK_HIV_DON_U','CLIN_INFECT_DON_U','DATA_TRANSPLANT_Y','DATA_WAITLIST_Y','DIABETES_DON_U','DIAL_TX_N','DIAL_TX_Y','EXTRACRANIAL_CANCER_DON_Y','HBV_CORE_DON.1_U','HBV_CORE_DON_I','HBV_CORE_ND','HBV_CORE_U','HBV_SUR_ANTIGEN_DON_C','HBV_SUR_ANTIGEN_DON_I','HCV_SEROSTATUS_ND','HEP_C_ANTI_DON_C','HEP_C_ANTI_DON_I','HISTORY_MI_DON_N','HIST_CANCER_DON_U','HIST_OTH_DRUG_DON_U','HIV_SEROSTATUS_ND','HIV_SEROSTATUS_P','INSULIN_DON_Y','LIFE_SUP_TRR_N','LIFE_SUP_TRR_Y','LIST_MELD_No','LT_ONE_WEEK_DON_N','MALIG_TRR_Y','PT_T3_DON_N','PT_T3_DON_Y','SKIN_CANCER_DON_N','TXLIV_S','TXLIV_W','TX_MELD_No','VDRL_DON_I']
cols = [col for col in result.columns if col not in less_important_features]
x_test= result[cols]


## Handle Imbalance in the class

In [20]:
y_train['GSTATUS'].value_counts()

0.0    27505
1.0    11657
Name: GSTATUS, dtype: int64

# SMOTE Algorithm for oversampling

sm = SMOTE()
x_train, y_train = sm.fit_sample(x_train, y_train)

print('Resampled dataset shape {}'.format(Counter(y_train)))
x_train= pd.DataFrame(x_train)
y_train= pd.DataFrame(y_train)


result =pd.concat([x_train,y_train],axis=1)
result.columns=headers

#display(result)

x_train = result[[column for column in result.columns if column != 'GSTATUS']]
y_train= result[['GSTATUS']]



# Random Forest

In [21]:
from sklearn.model_selection import StratifiedKFold
from sklearn import model_selection

kfold = StratifiedKFold(n_splits=10, shuffle=True)
cvscores = []
cv1scores = []
cv2scores = []
cv3scores= []

X = np.array(X)
Y = np.array(Y)

for i, (train, test) in enumerate(kfold.split(X,Y)):
       

    kfold = model_selection.KFold(n_splits=10, random_state=7)
    
    # oversampling
    sm = SMOTE()
    x_train, y_train = sm.fit_sample(X[train], Y[train])

    RF_model = RFC()
    RF_model.fit(x_train,y_train)
    y_pred = RF_model.predict(X[test])

    cvscores.append(metrics.precision_score(Y[test], y_pred,average='binary'))
    cv1scores.append(metrics.recall_score(Y[test], y_pred,average='binary'))
    cv2scores.append(metrics.f1_score(Y[test], y_pred,average='binary'))
    cv3scores.append(metrics.roc_auc_score(Y[test], y_pred))
    


print ('precision score: %.3f' % np.mean(cvscores))
print ('recall score: %.3f' % np.mean(cv1scores))
print ('f1 score: %.3f' % np.mean(cv2scores))
print ('auc roc score: %.3f' % np.mean(cv3scores))

precision score: 0.436
recall score: 0.204
f1 score: 0.278
auc roc score: 0.546


# Logistic Regression with Cross Validation

In [22]:
from sklearn.model_selection import StratifiedKFold
from sklearn import model_selection

kfold = StratifiedKFold(n_splits=10, shuffle=True)
cvscores = []
cv1scores = []
cv2scores = []
cv3scores= []

X = np.array(X)
Y = np.array(Y)

for i, (train, test) in enumerate(kfold.split(X,Y)):
       

    kfold = model_selection.KFold(n_splits=10, random_state=7)
    
    # oversampling
    sm = SMOTE()
    x_train, y_train = sm.fit_sample(X[train], Y[train])

    logreg = linear_model.LogisticRegression()
    logreg.fit(x_train, y_train)
    y_pred =logreg.predict(X[test])

    cvscores.append(metrics.precision_score(Y[test], y_pred,average='binary'))
    cv1scores.append(metrics.recall_score(Y[test], y_pred,average='binary'))
    cv2scores.append(metrics.f1_score(Y[test], y_pred,average='binary'))
    cv3scores.append(metrics.roc_auc_score(Y[test], y_pred))
    


print ('precision score: %.3f' % np.mean(cvscores))
print ('recall score: %.3f' % np.mean(cv1scores))
print ('f1 score: %.3f' % np.mean(cv2scores))
print ('auc roc score: %.3f' % np.mean(cv3scores))

precision score: 0.443
recall score: 0.634
f1 score: 0.521
auc roc score: 0.648


# Deep Learning

In [23]:
from sklearn.model_selection import StratifiedKFold
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers.normalization import BatchNormalization

input_neurons=int(X.shape[1])
kfold = StratifiedKFold(n_splits=10, shuffle=True)
cvscores = []
cv1scores = []
cv2scores = []
cv3scores= []

X = np.array(X)
Y = np.array(Y)

for i, (train, test) in enumerate(kfold.split(X,Y)):
    model = Sequential()
    model.add(Dense(input_neurons,input_dim=input_neurons, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(200, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(150, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(100, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(50, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(10, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(1,activation='sigmoid'))

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

    
    # oversampling
    sm = SMOTE()
    x_train, y_train = sm.fit_sample(X[train], Y[train])
    #Evaluate the Model
    model.fit(x_train, y_train, epochs=100, batch_size=120)
    y_pred=model.predict(X[test])
    y_pred = np.where(y_pred>0.5,1,0)
    cvscores.append(metrics.precision_score(Y[test], y_pred,average='binary'))
    cv1scores.append(metrics.recall_score(Y[test], y_pred,average='binary'))
    cv2scores.append(metrics.f1_score(Y[test], y_pred,average='binary'))
    cv3scores.append(metrics.roc_auc_score(Y[test], y_pred))
    


print ('precision score: %.3f' % np.mean(cvscores))
print ('recall score: %.3f' % np.mean(cv1scores))
print ('f1 score: %.3f' % np.mean(cv2scores))
print ('auc roc score: %.3f' % np.mean(cv3scores))

Epoch 1/100
49508/49508 [==============================] - 8s 156us/step - loss: 0.6243 - acc: 0.6639
Epoch 2/100
49508/49508 [==============================] - 6s 122us/step - loss: 0.5298 - acc: 0.7322
Epoch 3/100
49508/49508 [==============================] - 6s 121us/step - loss: 0.4998 - acc: 0.7524
Epoch 4/100
49508/49508 [==============================] - 6s 122us/step - loss: 0.4698 - acc: 0.7722
Epoch 5/100
49508/49508 [==============================] - 6s 121us/step - loss: 0.4430 - acc: 0.7897
Epoch 6/100
49508/49508 [==============================] - 6s 122us/step - loss: 0.4123 - acc: 0.8065
Epoch 7/100
49508/49508 [==============================] - 6s 121us/step - loss: 0.3821 - acc: 0.8267
Epoch 8/100
49508/49508 [==============================] - 7s 150us/step - loss: 0.3558 - acc: 0.8398
Epoch 9/100
49508/49508 [==============================] - 8s 163us/step - loss: 0.3313 - acc: 0.8537
Epoch 10/100
49508/49508 [==============================] - 8s 159us/step - loss: 

49508/49508 [==============================] - 6s 116us/step - loss: 0.0652 - acc: 0.9762
Epoch 53/100
49508/49508 [==============================] - 6s 122us/step - loss: 0.0634 - acc: 0.9762
Epoch 54/100
49508/49508 [==============================] - 6s 113us/step - loss: 0.0660 - acc: 0.9757
Epoch 55/100
49508/49508 [==============================] - 6s 120us/step - loss: 0.0586 - acc: 0.9780 3s -
Epoch 56/100
49508/49508 [==============================] - 6s 120us/step - loss: 0.0611 - acc: 0.9767
Epoch 57/100
49508/49508 [==============================] - 6s 130us/step - loss: 0.0605 - acc: 0.9770
Epoch 58/100
49508/49508 [==============================] - 6s 125us/step - loss: 0.0592 - acc: 0.9781
Epoch 59/100
49508/49508 [==============================] - 6s 118us/step - loss: 0.0604 - acc: 0.9771
Epoch 60/100
49508/49508 [==============================] - 6s 127us/step - loss: 0.0608 - acc: 0.9774 1s - loss: 0.0 - ETA: 0s - loss: 0.0601 - acc: 
Epoch 61/100
49508/49508 [=======

49508/49508 [==============================] - 6s 112us/step - loss: 0.3053 - acc: 0.8669
Epoch 11/100
49508/49508 [==============================] - 6s 113us/step - loss: 0.2823 - acc: 0.8775
Epoch 12/100
49508/49508 [==============================] - 6s 114us/step - loss: 0.2636 - acc: 0.8878
Epoch 13/100
49508/49508 [==============================] - 6s 112us/step - loss: 0.2511 - acc: 0.8932
Epoch 14/100
49508/49508 [==============================] - 6s 112us/step - loss: 0.2244 - acc: 0.9064
Epoch 15/100
49508/49508 [==============================] - 6s 112us/step - loss: 0.2128 - acc: 0.9120
Epoch 16/100
49508/49508 [==============================] - 6s 117us/step - loss: 0.1960 - acc: 0.9188
Epoch 17/100
49508/49508 [==============================] - 6s 116us/step - loss: 0.1859 - acc: 0.9248
Epoch 18/100
49508/49508 [==============================] - 6s 115us/step - loss: 0.1773 - acc: 0.9281
Epoch 19/100
49508/49508 [==============================] - 6s 121us/step - loss: 0.16

49508/49508 [==============================] - 6s 118us/step - loss: 0.0488 - acc: 0.9821
Epoch 68/100
49508/49508 [==============================] - 6s 120us/step - loss: 0.0544 - acc: 0.9795
Epoch 69/100
49508/49508 [==============================] - 6s 120us/step - loss: 0.0530 - acc: 0.9809
Epoch 70/100
49508/49508 [==============================] - 6s 119us/step - loss: 0.0542 - acc: 0.9802
Epoch 71/100
49508/49508 [==============================] - 6s 122us/step - loss: 0.0507 - acc: 0.9820
Epoch 72/100
49508/49508 [==============================] - 6s 119us/step - loss: 0.0485 - acc: 0.9822
Epoch 73/100
49508/49508 [==============================] - 5s 105us/step - loss: 0.0490 - acc: 0.9816
Epoch 74/100
49508/49508 [==============================] - 4s 85us/step - loss: 0.0533 - acc: 0.9801
Epoch 75/100
49508/49508 [==============================] - 4s 85us/step - loss: 0.0494 - acc: 0.9817
Epoch 76/100
49508/49508 [==============================] - 4s 82us/step - loss: 0.0506 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\msros\anaconda22\envs\py36\lib\site-packages\IPython\core\interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-23-bbd2ec1e2039>", line 40, in <module>
    model.fit(x_train, y_train, epochs=100, batch_size=120)
  File "c:\users\msros\anaconda22\envs\py36\lib\site-packages\keras\models.py", line 963, in fit
    validation_steps=validation_steps)
  File "c:\users\msros\anaconda22\envs\py36\lib\site-packages\keras\engine\training.py", line 1705, in fit
    validation_steps=validation_steps)
  File "c:\users\msros\anaconda22\envs\py36\lib\site-packages\keras\engine\training.py", line 1221, in _fit_loop
    ins_batch = _slice_arrays(ins[:-1], batch_ids) + [ins[-1]]
  File "c:\users\msros\anaconda22\envs\py36\lib\site-packages\keras\engine\training.py", line 385, in _slice_arrays
    return [None if x is None else x[start] for x in arrays]
  File "c:\users\msros\anaconda22

KeyboardInterrupt: 

# Save Random Forest, Logistic Regression and Keras Model into files

In [ ]:
#Save and write the Random Forest model
rfmodel_pkl = open('rfmodel.pkl', 'wb')
pickle.dump(RF_model, rfmodel_pkl)
rfmodel_pkl.close()

#Save and write the Logistic Regression model
lrmodel_pkl = open('lrmodel.pkl', 'wb')
pickle.dump(logreg, lrmodel_pkl)
lrmodel_pkl.close()

#Save and write the Deep Learning model
model.save('dlmodel.h5')

# Ensemble Method

In [ ]:
#Load Random Forest
rfmodel_pkl = open('rfmodel.pkl', 'rb')
clf1_rf = pickle.load(rfmodel_pkl)
y_pred_clf1 = clf1_rf.predict_proba(x_test)
y_pred_1 = np.array(y_pred_clf1)
y_pred_1 = np.delete(y_pred_1,np.s_[:1],1)

#Load Logistic Regression
lrmodel_pkl = open('lrmodel.pkl', 'rb')
clf2_lr = pickle.load(lrmodel_pkl)
y_pred_clf2 = clf2_lr.predict_proba(x_test)
y_pred_2 = np.array(y_pred_clf2)
y_pred_2 = np.delete(y_pred_2,np.s_[:1],1)

#Load Deep Learning Model
clf3_dl = load_model('dlmodel.h5')
y_pred_3=clf3_dl.predict(x_test)
y_pred_3 = np.array(y_pred_3)


In [ ]:
ens_arr = np.concatenate((y_pred_1,y_pred_2,y_pred_3),axis=1)
ens_arr = np.mean(ens_arr,axis=1)
pred_ens = np.where(ens_arr>0.5,1,0)
print ('Accuracy score: %.3f' % metrics.accuracy_score(y_test, pred_ens))
print('precision score: %.3f' % metrics.precision_score(y_test, pred_ens))
print('recall score: %.3f' % metrics.recall_score(y_test, pred_ens))
print('f1 score: %.3f' % metrics.f1_score(y_test,pred_ens))
print ('roc_auc score: %.3f' % metrics.roc_auc_score(y_test, pred_ens))
